In [1]:
from src.Dataset import getDataLoader
import torch
from src.utils import *

In [2]:
classes = ["4ask","8pam","16psk","32qam_cross","2fsk","ofdm-256"]
iq_samples = 1024
samples_per_class= 1000
batch_size = 32
epochs = 10
eff_net_PATH = f'src/SavedModels/efficientNet_epochs_{epochs}.pt'
in_channels = 2

dl_train, ds_train, dl_test, ds_test, dl_val, ds_val = getDataLoader(
    classes = classes,
    iq_samples = iq_samples,
    samples_per_class= samples_per_class,
    batch_size=batch_size
    )

In [3]:
ds_train[0][0].shape

(2, 1024)

In [4]:
next(iter(dl_train))[0].shape

torch.Size([32, 2, 1024])

In [5]:
vqvae_model = torch.load('src/SavedModels/VQVAE_epochs_10.pt').to(device)

/tmp/ipykernel_2459829/3239689431.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vqvae_model = torch.load('src/SavedModels/VQVAE_epochs_10.pt').to(device)


In [ ]:
test_data = torch.cat([torch.cat((label.unsqueeze(1).to(device), vqvae_model.codebook.quantize_indices(vqvae_model.encode(x.to(device)))), dim=1) for x, label in dl_test],dim=0)

In [13]:
model_transformer = torch.load('src/SavedModels/MonaiTransformer_epochs_10.pt').to(device)
#model_transformer = torch.load('src/SavedModels/Transformer_epochs_10.pt').to(device)

/tmp/ipykernel_2459829/3371109386.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_transformer = torch.load('src/SavedModels/MonaiTransformer_epochs_10.pt').to(devi

In [19]:
context = torch.tensor([[3]]).to(device)
sample1 = model_transformer.generate(context, max_new_tokens=513)
sample2 = model_transformer.generate(context, max_new_tokens=513)

In [20]:
print(sample1[:,:15])
print(sample2[:,:15])

tensor([[ 3, 23,  7, 33, 57,  5, 44,  7,  5, 61,  9, 13, 11, 43, 21]],
       device='cuda:1')
tensor([[ 3, 24, 16, 32, 20,  4, 33, 13, 32,  9,  3,  3, 21, 23, 33]],
       device='cuda:1')


In [16]:
difference = sample1 == sample2 # Element-wise comparison
print("Element-wise comparison result:")
print(difference)

if torch.all(difference):
    print("The tensors are exactly the same.")
else:
    print("The tensors are different.")


Element-wise comparison result:
tensor([[ True,  True, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False,  True, False,
         False, False, False, False, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,  True,
         False, False, False, False, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, Fals